I'd like to train VGG network as example, but it just ran too slow on my computer.

I have only mac, so I can only run tensorflow with CPU :(

So, I refer to our good old friend, handwriting recognition with MNIST.

Bear with me, plz. I just want to show you a full process converting a graph to a single reusable file.

In [1]:
from math import sqrt
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist.input_data import read_data_sets

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
# helper function
def accuracy(y, y_):
    return (y == y_).mean()

In [3]:
mnist = read_data_sets("mnist_data", one_hot=True)

Extracting mnist_data/train-images-idx3-ubyte.gz
Extracting mnist_data/train-labels-idx1-ubyte.gz
Extracting mnist_data/t10k-images-idx3-ubyte.gz
Extracting mnist_data/t10k-labels-idx1-ubyte.gz


In [4]:
# build a trainable graph
train_graph = tf.Graph()
hidden_shape = [100, 100]
learning_rate = 0.08

with train_graph.as_default():
    with tf.name_scope("input"):
        image_batch = tf.placeholder(tf.float32, 
                                     shape=[None, 784],
                                     name="image_batch")
    layer_shapes = [784] + hidden_shape + [10]
    last_layer = image_batch
    with tf.name_scope("hidden"):
        for in_shape, out_shape in zip(layer_shapes[:-1], layer_shapes[1:]):
            W = tf.Variable(np.random.randn(in_shape, out_shape),
                            dtype=tf.float32,
                            name="weight")
            bias = tf.Variable(np.random.randn(out_shape),
                               dtype=tf.float32,
                               name="bias")
            zscore = tf.matmul(last_layer, W) + bias
            last_layer = tf.nn.sigmoid(zscore)
    
    with tf.name_scope("output"):
        prob = tf.nn.softmax(zscore, 
                             name="probability")
        predict = tf.arg_max(prob, 
                             1, 
                             name="prediction")
        target = tf.placeholder(tf.float32,
                                shape=[None, 10],
                                name="target")
    loss = -tf.reduce_mean(target*tf.log(prob))
    train_op = tf.train.AdagradOptimizer(learning_rate).minimize(loss)
    # when saver is constructed, it will colloect all savable variables in default graph
    saver = tf.train.Saver()

In [5]:
# Training and save
num_iters = 20000
batch_size = 200

with tf.Session(graph=train_graph) as sess:
    tf.global_variables_initializer().run()
    print("Initialized")
    
    for step in range(num_iters):
        images, labels = mnist.train.next_batch(batch_size)
        feed_dict = { image_batch: images,
                      target: labels }
        l, _ = sess.run([loss, train_op], feed_dict=feed_dict)
        
        if (step+1) % 1000 == 0:
            pred = sess.run(predict, feed_dict={image_batch:mnist.test.images})
            acc = accuracy(pred, np.argmax(mnist.test.labels, axis=1))
            print("Iteration {}: {:.4f} {:.2f}%".format(step+1, l, acc*100))
    
    saver.save(sess, save_path="model/mnist_example.chkp", global_step=step)

Initialized
Iteration 1000: 0.1344 59.35%
Iteration 2000: 0.0955 70.11%
Iteration 3000: 0.0692 75.14%
Iteration 4000: 0.0752 77.97%
Iteration 5000: 0.0553 79.94%
Iteration 6000: 0.0496 81.28%
Iteration 7000: 0.0562 82.40%
Iteration 8000: 0.0525 83.35%
Iteration 9000: 0.0494 84.10%
Iteration 10000: 0.0614 84.86%
Iteration 11000: 0.0408 85.26%
Iteration 12000: 0.0368 85.72%
Iteration 13000: 0.0458 86.13%
Iteration 14000: 0.0443 86.49%
Iteration 15000: 0.0456 86.81%
Iteration 16000: 0.0370 87.03%
Iteration 17000: 0.0425 87.46%
Iteration 18000: 0.0453 87.59%
Iteration 19000: 0.0447 87.94%
Iteration 20000: 0.0420 88.17%


Looking good enough, ready to save the graph

In [6]:
# saved data
!ls model/

checkpoint
mnist_example.chkp-19999.data-00000-of-00001
mnist_example.chkp-19999.index
mnist_example.chkp-19999.meta


In [7]:
# import the graph_util
# We'll use it to convert all variables in graph to constants
from tensorflow.python.framework import graph_util

In [8]:
# A CheckpointState object from tensorflow.python.training.checkpoint_state_pb2
# Yes, it's produced by protobuf.
checkpoint = tf.train.get_checkpoint_state("model/")
checkpoint.model_checkpoint_path

'model/mnist_example.chkp-19999'

In [9]:
# meta file contains the meta data of the saved session
meta_file_path = checkpoint.model_checkpoint_path+".meta"
meta_file_path

'model/mnist_example.chkp-19999.meta'

In [10]:
# restore the session with restored saver
# convert graph to "freezed" graph
# Note that you need to specify the layers you want to output, 
# tensorflow will extract a subgraph which involves the layers you want and freeze it.
restore_graph = tf.Graph()
with restore_graph.as_default():
    # reconstruct graph
    saver = tf.train.import_meta_graph(meta_file_path, clear_devices=True)
    restore_graph_def = restore_graph.as_graph_def() # get graph_def of the reconstructed graph
with tf.Session(graph=restore_graph) as sess:
    saver.restore(sess, checkpoint.model_checkpoint_path)
    # convert variables to constants
    # this step is necessary since otherwise you have to initialize the variables later
    # in the restored graph or those variables will remain uninitialized.
    out_graph_def = graph_util.convert_variables_to_constants(sess,
                                                              restore_graph_def,
                                                              [predict.op.name])

INFO:tensorflow:Restoring parameters from model/mnist_example.chkp-19999
INFO:tensorflow:Froze 6 variables.
Converted 6 variables to const ops.


In [11]:
# write graph to disk just like what you will do with any protobuf object
with tf.gfile.GFile("my_mnist.pb", "wb") as fid:
    fid.write(out_graph_def.SerializeToString())

In [12]:
from serialization_utils import chkp_to_freeze_graph

In [13]:
freeze_def = chkp_to_freeze_graph("model/mnist_example.chkp-19999",
                                  [predict.op.name],
                                  True,
                                  "mnist.pb")

INFO:tensorflow:Restoring parameters from model/mnist_example.chkp-19999
INFO:tensorflow:Froze 6 variables.
Converted 6 variables to const ops.


In [14]:
for node in freeze_def.node:
    print(node.name, node.op)
    if node.attr["shape"]:
        print(node.attr["shape"])

input/image_batch Placeholder
shape {
  dim {
    size: -1
  }
  dim {
    size: 784
  }
}

hidden/weight Const

hidden/weight/read Identity

hidden/bias Const

hidden/bias/read Identity

hidden/MatMul MatMul

hidden/add Add

hidden/Sigmoid Sigmoid

hidden/weight_1 Const

hidden/weight_1/read Identity

hidden/bias_1 Const

hidden/bias_1/read Identity

hidden/MatMul_1 MatMul

hidden/add_1 Add

hidden/Sigmoid_1 Sigmoid

hidden/weight_2 Const

hidden/weight_2/read Identity

hidden/bias_2 Const

hidden/bias_2/read Identity

hidden/MatMul_2 MatMul

hidden/add_2 Add

output/probability Softmax

output/prediction/dimension Const

output/prediction ArgMax



------

Testing time!

In [15]:
graph = tf.Graph()
graph_def = graph.as_graph_def()

In [16]:
with tf.gfile.GFile("mnist.pb", "rb") as rf:
    graph_def.ParseFromString(rf.read())
with graph.as_default():
    tf.import_graph_def(graph_def, name="")

In [17]:
images_tensor = graph.get_tensor_by_name("input/image_batch:0")
predict_tensor = graph.get_tensor_by_name("output/prediction:0")

In [18]:
with tf.Session(graph=graph) as sess:
#     tf.global_variables_initializer().run()
    pred = sess.run(predict_tensor, 
                    feed_dict={images_tensor:mnist.test.images})
    print(accuracy(np.argmax(mnist.test.labels, axis=1), pred)*100)

88.17
